<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/wikiWord2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##学習済みDoc2Vecモデル
https://yag-ays.github.io/project/pretrained_doc2vec_wikipedia/

ライブラリ
Gensim: 3.6.0

In [68]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from gensim.models import KeyedVectors
from pprint import pprint

#Gドライブ保存後の確認
model_dir = './drive/My Drive/NLP/wiki_tohoku.model'
model = KeyedVectors.load(model_dir)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [3]:
!pip install mecab-python3

     |████████████████████████████████| 15.9MB 3.6MB/s 


In [0]:
import MeCab

def search_similar_texts(p_words):
  try:
    print(model.similarity('[ポケモン]',p_words))
  except KeyError as error:
    word_cng_list.append(cut_wd.replace('[',"").replace(']',""))

words = '君の名は'
inherent_words = '[' + words + ']'
word_cng_list = []

#**************************************************************************
#そのまま対義化
#**************************************************************************
asitis_cng_words = ''
try:
  for i in range(5):
    rvs_wd = model.most_similar(positive=[inherent_words,u"逆"])[i][0]
    if inherent_words.replace('[',"").replace(']',"") != rvs_wd.replace('[',"").replace(']',""):
      asitis_cng_words = model.most_similar(positive=[inherent_words,u"逆"])[i][0]
      break
    else:
      asitis_cng_words = words + "(変換なし)"
except KeyError as error:
  #辞書に登録の無い単語の場合
  search_similar_texts(inherent_words)

#**************************************************************************
#形態素分析後に対義化
#**************************************************************************
#tokenizer = MeCab.Tagger("-Ochasen")
node = tokenizer.parseToNode(words)
  
while node:
  cut_wd = node.surface

  if node.feature.split(",")[0] == u"名詞":
    try:
      for i in range(5):
        rvs_wd = model.most_similar(positive=[cut_wd,u"逆"])[i][0]
        if cut_wd != rvs_wd.replace('[',"").replace(']',""):
          #◆結合
          word_cng_list.append(rvs_wd.replace('[',"").replace(']',""))
          break
    except KeyError as error:
      print("名詞キーエラー")
      #辞書に登録の無い単語の場合
      search_similar_texts(cut_wd)

  elif node.feature.split(",")[0] == u"形容詞":
    cut_wd = node.feature.split(",")[6]

    try:
      for i in range(5):
        rvs_wd = model.most_similar(positive=[cut_wd,u"逆"])[i][0]
        if cut_wd != rvs_wd.replace('[',"").replace(']',""):
          #◆結合
          word_cng_list.append(rvs_wd.replace('[',"").replace(']',""))
          break
    except KeyError as error:
      print("形容詞キーエラー")
      #辞書に登録の無い単語の場合
      search_similar_texts(cut_wd)

  elif node.feature.split(",")[0] == u"動詞":
    cu_wd = node.feature.split(",")[6]

    try:
      for i in range(5):
        rvs_wd = model.most_similar(positive=[cut_wd,u"逆"])[i][0]
        if cut_wd != rvs_wd.replace('[',"").replace(']',""):
          #◆結合
          word_cng_list.append(rvs_wd.replace('[',"").replace(']',""))
          break
    except KeyError as error:
      print("動詞キーエラー")
      #辞書に登録の無い単語の場合
      search_similar_texts(cut_wd)

  else:
    #◆結合
    word_cng_list.append(cut_wd.replace('[',"").replace(']',""))

  node = node.next

print(words)
print(asitis_cng_words)
print(word_cng_list)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


君の名は
[あぶない刑事]
['', 'お前', 'の', '首', 'は', '']


In [69]:
import pandas as pd
import itertools
#**************************************************************************
#分類特定
#**************************************************************************
max_cate = ""

is_a_parent = pd.read_table('./drive/My Drive/NLP/is_a_parent_list.txt')

#dataframeをlist化
is_a_parent_list = is_a_parent.values.tolist()
#2次元listから1次元listへ
is_a_parent_list = list(itertools.chain.from_iterable(is_a_parent_list))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


In [0]:
#r'文字列'　→　正規表現。
#u'文字列'　→　日本語。Ver.2.7のとき必要。Ver.3では付けても付けなくてもよい。
#  pprint(model.most_similar(positive=[u'独立'],negative=[u"依存"],topn=2))

def group_check(child_wd):

  print("word:",child_wd)

  #is-a関係の判別
  for is_a_word in is_a_parent_list:
    try:
      parent_wd = is_a_word
      isa_val_list = []

      for i in range(5):
        rvs_wd = model.most_similar(positive=[child_wd])[i][0]
        if child_wd != rvs_wd.replace('[',"").replace(']',""):
          isa_val_list.append(model.similarity(parent_wd,rvs_wd))

          for ii in range(5):
            rvs_wd2 = model.most_similar(positive=[rvs_wd])[ii][0]
            if rvs_wd != rvs_wd2.replace('[',"").replace(']',""):
              isa_val_list.append(model.similarity(parent_wd,rvs_wd2))

    except KeyError as error:
      print(error)

    ave_val = sum(isa_val_list) / len(isa_val_list)
    #max_val = max(isa_val_list)
    #min_val = min(isa_val_list)

    print("【",parent_wd,"】is-a:",ave_val)
    # print("最大値:",max_val)
    # print("最小値:",min_val)

    #類似度が0.5以上の場合はカテゴリ特定と見なし処理終了
    if ave_val >= 0.5:
      break


In [64]:
import MeCab
cnt = 0

del_list =['(',')','-','"','、','。','/','.',',',':',')。',')、','「','」','『','』','()','』(','=','～','+','']

tokenizer = MeCab.Tagger("-Ochasen")
#TODO:フィラーや記号は排除

for k, v in model.vocab.items():

  if not k in del_list:
    node = tokenizer.parseToNode(k)
    while node:
      if node.feature.split(",")[0] == u"名詞":
        if(node.feature.split(",")[1] == u"サ変接続" or
           node.feature.split(",")[1] == u"一般" or
           node.feature.split(",")[1] == u"固有名詞"):
          #★グループ判別
          group_check(k)
          cnt = cnt + 1
      else:
        pass
      node = node.next

      if cnt > 5:
        break


word: もの


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


【 男性 】is-a: 0.13246950536655883


KeyboardInterrupt: ignored

In [42]:
'l' in del_list

False

In [12]:
text = tokenizer.parse('こと')
print(text)

こと	コト	こと	名詞-非自立-一般		
EOS



In [6]:
cnt = 0
for k, v in model.vocab.items():
  print(k)
  cnt = cnt + 1
  if cnt > 100:
    break
  

の
、
。
に
は
を
た
が
で
て
と
し
(
れ
さ
)
ある
いる
する
も
「
・
」
から
こと
な
として
い
や
なっ
れる
など
『
この
ない
-
』
ため
あっ
その
的
また
まで
よう
へ
られ
あり
なる
という
後
もの
2
1
第
"
中
だ
おり
により
これ
より
によって
,
)、
現在
による
)。
3
.
者
なり
だっ
ず
か
において
なかっ
られる
場合
時
行わ
その後
[日本]
多く
なく
について
名
せ
しかし
ば
間
4
できる
それ
上
使用
内
日本
化
他
作品
放送


In [0]:
print(model.similarity('動物','[イヌ]'))


0.70940584


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
